In [ ]:
!pip install rasterio patchify torch torchvision matplotlib numpy tqdm scikit-learn

In [ ]:
import os, math
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
import rasterio
from rasterio.plot import reshape_as_image

# === Paths ===
data_dir = "../data/"
save_dir = "../model/"
os.makedirs(save_dir, exist_ok=True)

# === Load image & mask (2024 only) ===
def load_tif_as_array(filepath):
    with rasterio.open(filepath) as src:
        img = src.read().astype(np.float32)
        img = reshape_as_image(img)
    return img

def normalize_image(img):
    return img / np.max(img)

def binarize_mask(mask):
    mask = mask[:, :, 0]
    if np.max(mask) <= 0.01:
        mask = (mask > 0.001).astype(np.uint8)
    elif np.max(mask) > 1:
        mask = (mask > 128).astype(np.uint8)
    else:
        mask = (mask > 0.5).astype(np.uint8)
    return mask

# === Load 2024 data ===
img_2024 = normalize_image(load_tif_as_array(
    data_dir + "20241110_053942_45_24f7_3B_AnalyticMS_SR_8b_clip.tif"
))
mask_2024 = binarize_mask(load_tif_as_array(
    data_dir + "20241110_053942_45_24f7_3B_AnalyticMS_SR_8b_clip_Hybrid_mask.tif"
))
print("✅ 2024 image & mask loaded:", img_2024.shape, mask_2024.shape)

# === Patch extraction ===
def extract_patches(img, mask, patch_size=256, stride=256):
    X, Y = [], []
    H, W, _ = img.shape
    for i in range(0, H - patch_size, stride):
        for j in range(0, W - patch_size, stride):
            patch_i = img[i:i+patch_size, j:j+patch_size, :]
            patch_m = mask[i:i+patch_size, j:j+patch_size]
            if np.sum(patch_m) > 0:  # only patches with some landslide
                X.append(patch_i)
                Y.append(patch_m)
    X, Y = np.array(X), np.array(Y)
    Y = np.expand_dims(Y, -1)
    return X, Y

X, y = extract_patches(img_2024, mask_2024)
print("✅ Patch dataset:", X.shape, y.shape)

# === Split dataset ===
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train patches:", X_train.shape, "Validation patches:", X_val.shape)

# === Hyperparameters ===
EPOCHS = 50
LR = 5e-4
tf.config.run_functions_eagerly(True)

# === Helper blocks ===
def residual_block(x, filters):
    shortcut = layers.Conv2D(filters, (1,1), padding='same')(x)
    x = layers.Conv2D(filters, (3,3), padding='same', activation='relu')(x)
    x = layers.Conv2D(filters, (3,3), padding='same')(x)
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def attention_block(x, g, filters):
    g = layers.UpSampling2D(size=(2,2), interpolation='bilinear')(g)
    theta_x = layers.Conv2D(filters, (1,1), padding='same')(x)
    phi_g = layers.Conv2D(filters, (1,1), padding='same')(g)
    add_xg = layers.Add()([theta_x, phi_g])
    act_xg = layers.Activation('relu')(add_xg)
    psi = layers.Conv2D(1, (1,1), padding='same', activation='sigmoid')(act_xg)
    return layers.Multiply()([x, psi])

# === Model 1: U-Net ===
def build_unet(input_shape=(256,256,8)):
    inputs = layers.Input(input_shape)
    c1 = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D()(c1)

    c2 = layers.Conv2D(64, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D()(c2)

    c3 = layers.Conv2D(128, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(128, 3, activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D()(c3)

    c4 = layers.Conv2D(256, 3, activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(256, 3, activation='relu', padding='same')(c4)

    u5 = layers.UpSampling2D()(c4)
    u5 = layers.Concatenate()([u5, c3])
    c5 = layers.Conv2D(128, 3, activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(128, 3, activation='relu', padding='same')(c5)

    u6 = layers.UpSampling2D()(c5)
    u6 = layers.Concatenate()([u6, c2])
    c6 = layers.Conv2D(64, 3, activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(64, 3, activation='relu', padding='same')(c6)

    u7 = layers.UpSampling2D()(c6)
    u7 = layers.Concatenate()([u7, c1])
    c7 = layers.Conv2D(32, 3, activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(32, 3, activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c7)
    return models.Model(inputs, outputs)

# === Model 2: ResU-Net ===
def build_resunet(input_shape=(256,256,8)):
    inputs = layers.Input(input_shape)
    c1 = residual_block(inputs, 32)
    p1 = layers.MaxPooling2D()(c1)
    c2 = residual_block(p1, 64)
    p2 = layers.MaxPooling2D()(c2)
    c3 = residual_block(p2, 128)
    p3 = layers.MaxPooling2D()(c3)
    c4 = residual_block(p3, 256)

    u5 = layers.UpSampling2D()(c4)
    u5 = layers.Concatenate()([u5, c3])
    c5 = residual_block(u5, 128)
    u6 = layers.UpSampling2D()(c5)
    u6 = layers.Concatenate()([u6, c2])
    c6 = residual_block(u6, 64)
    u7 = layers.UpSampling2D()(c6)
    u7 = layers.Concatenate()([u7, c1])
    c7 = residual_block(u7, 32)
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c7)
    return models.Model(inputs, outputs)

# === Model 3: Attention U-Net ===
def build_att_unet(input_shape=(256,256,8)):
    inputs = layers.Input(input_shape)
    c1 = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    p1 = layers.MaxPooling2D()(c1)
    c2 = layers.Conv2D(64, 3, activation='relu', padding='same')(p1)
    p2 = layers.MaxPooling2D()(c2)
    c3 = layers.Conv2D(128, 3, activation='relu', padding='same')(p2)
    p3 = layers.MaxPooling2D()(c3)
    c4 = layers.Conv2D(256, 3, activation='relu', padding='same')(p3)
    g1 = attention_block(c3, c4, 128)
    u1 = layers.UpSampling2D()(c4)
    u1 = layers.Concatenate()([u1, g1])
    c5 = layers.Conv2D(128, 3, activation='relu', padding='same')(u1)
    c5 = layers.Conv2D(128, 3, activation='relu', padding='same')(c5)
    u2 = layers.UpSampling2D()(c5)
    u2 = layers.Concatenate()([u2, c2])
    c6 = layers.Conv2D(64, 3, activation='relu', padding='same')(u2)
    c6 = layers.Conv2D(64, 3, activation='relu', padding='same')(c6)
    u3 = layers.UpSampling2D()(c6)
    u3 = layers.Concatenate()([u3, c1])
    c7 = layers.Conv2D(32, 3, activation='relu', padding='same')(u3)
    c7 = layers.Conv2D(32, 3, activation='relu', padding='same')(c7)
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c7)
    return models.Model(inputs, outputs)

# === Model 4: Attention ResU-Net ===
def build_att_resunet(input_shape=(256,256,8)):
    inputs = layers.Input(input_shape)
    c1 = residual_block(inputs, 32)
    p1 = layers.MaxPooling2D()(c1)
    c2 = residual_block(p1, 64)
    p2 = layers.MaxPooling2D()(c2)
    c3 = residual_block(p2, 128)
    p3 = layers.MaxPooling2D()(c3)
    c4 = residual_block(p3, 256)
    g1 = attention_block(c3, c4, 128)
    u1 = layers.UpSampling2D()(c4)
    u1 = layers.Concatenate()([u1, g1])
    c5 = residual_block(u1, 128)
    u2 = layers.UpSampling2D()(c5)
    u2 = layers.Concatenate()([u2, c2])
    c6 = residual_block(u2, 64)
    u3 = layers.UpSampling2D()(c6)
    u3 = layers.Concatenate()([u3, c1])
    c7 = residual_block(u3, 32)
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c7)
    return models.Model(inputs, outputs)

# === Model 5: ASDMS U-Net ===
def build_asdms_unet(input_shape=(256,256,8)):
    base = build_unet(input_shape)
    return models.Model(base.input, base.output)

# === Train-and-save helper ===
def train_and_save(model, name):
    tf.keras.backend.clear_session()
    optimizer_local = optimizers.Adam(learning_rate=LR)
    model.compile(optimizer=optimizer_local, loss='binary_crossentropy', metrics=['accuracy'])
    print(f"\n🚀 Training {name} on 2024 data ... (50 epochs)")
    hist = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=8,
        verbose=1
    )
    model.save(os.path.join(save_dir, f"{name}_2022.keras"))
    print(f"✅ Saved {name}_2022.keras\n")
    return hist

# === Build and train all models ===
models_to_train = {
    "unet": build_unet(),
    "resunet": build_resunet(),
    "att_unet": build_att_unet(),
    "att_resunet": build_att_resunet(),
    "asdms_unet": build_asdms_unet()
}

histories_2024 = {}
for name, model in models_to_train.items():
    histories_2024[name] = train_and_save(model, name)


In [ ]:
import os, math
import numpy as np
import tensorflow as tf
import rasterio
from rasterio.plot import reshape_as_image
import matplotlib.pyplot as plt

# === Paths ===
data_dir = "../data/"
model_dir = "/model/"
os.makedirs(model_dir, exist_ok=True)

# === Load 2022 image ===
def load_tif_as_array(filepath):
    with rasterio.open(filepath) as src:
        img = src.read().astype(np.float32)
        img = reshape_as_image(img)
    return img

def normalize_image(img):
    return img / np.max(img)

img_2022 = normalize_image(load_tif_as_array(
    data_dir + "20220503_050008_52_2474_3B_AnalyticMS_8b_clip.tif"
))
print("✅ Loaded 2024 image:", img_2022.shape)

# === Patch-based prediction ===
def predict_large_image(model, image, patch_size=256):
    H, W, C = image.shape
    out_mask = np.zeros((H, W), dtype=np.uint8)
    n_rows = math.ceil(H / patch_size)
    n_cols = math.ceil(W / patch_size)

    for i in range(n_rows):
        for j in range(n_cols):
            y0, y1 = i * patch_size, min((i + 1) * patch_size, H)
            x0, x1 = j * patch_size, min((j + 1) * patch_size, W)
            patch = image[y0:y1, x0:x1, :]
            pad_h, pad_w = patch_size - patch.shape[0], patch_size - patch.shape[1]
            patch_padded = np.pad(patch, ((0, pad_h), (0, pad_w), (0, 0)), mode='reflect')
            pred = model.predict(np.expand_dims(patch_padded, axis=0), verbose=0)
            pred_mask = (pred[0, :, :, 0] > 0.5).astype(np.uint8)
            out_mask[y0:y1, x0:x1] = pred_mask[:patch.shape[0], :patch.shape[1]]
    return out_mask

# === Visualization + Save (No Ground Truth) ===
def visualize_and_save_prediction(image, pred_mask, model_name, save_dir):
    rgb = (image[:, :, :3] - np.min(image[:, :, :3])) / (np.max(image[:, :, :3]) - np.min(image[:, :, :3]))
    plt.figure(figsize=(12, 6))

    # Satellite
    plt.subplot(1, 2, 1)
    plt.imshow(rgb)
    plt.title("Satellite Image")
    plt.axis('off')

    # Predicted mask
    plt.subplot(1, 2, 2)
    plt.imshow(pred_mask, cmap='gray')
    plt.title(f"Predicted Mask ({model_name})")
    plt.axis('off')

    plt.tight_layout()

    save_path = os.path.join(save_dir, f"{model_name}_2022_prediction.png")
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"✅ Saved {model_name} prediction → {save_path}")

# === Model list ===
model_names = [
    "unet_2022.keras",
    "resunet_2022.keras",
    "att_unet_2022.keras",
    "att_resunet_2022.keras",
    "asdms_unet_2022.keras"
]

# === Test & Save All ===
for mname in model_names:
    path = os.path.join(model_dir, mname)
    if os.path.exists(path):
        print(f"\n🚀 Testing {mname} ...")
        model = tf.keras.models.load_model(path, compile=False)
        pred_mask = predict_large_image(model, img_2022)
        visualize_and_save_prediction(img_2022, pred_mask, mname.replace("_2022.keras", ""), model_dir)
    else:
        print(f"⚠️ Model not found: {path}")


In [ ]:
import os, math
import numpy as np
import tensorflow as tf
import rasterio
from rasterio.plot import reshape_as_image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

# === Paths ===
data_dir = "../data/"
model_dir = "../model/"
os.makedirs(model_dir, exist_ok=True)

# === Load 2022 image + mask ===
def load_tif_as_array(filepath):
    with rasterio.open(filepath) as src:
        img = src.read().astype(np.float32)
        img = reshape_as_image(img)
    return img

def normalize_image(img):
    return img / np.max(img)

def binarize_mask(mask):
    mask = mask[:, :, 0]
    if np.max(mask) <= 0.01:
        mask = (mask > 0.001).astype(np.uint8)
    elif np.max(mask) > 1:
        mask = (mask > 128).astype(np.uint8)
    else:
        mask = (mask > 0.5).astype(np.uint8)
    return mask

img_2022 = normalize_image(load_tif_as_array(
    data_dir + "20220503_050008_52_2474_3B_AnalyticMS_8b_clip.tif"
))
mask_2022 = binarize_mask(load_tif_as_array(
    data_dir + "landslide_mask_20220503_final.tif"
))
print("✅ Loaded 2022 data:", img_2022.shape, mask_2022.shape)

# === Patch-based prediction ===
def predict_large_image(model, image, patch_size=256):
    H, W, C = image.shape
    out_mask = np.zeros((H, W), dtype=np.uint8)
    n_rows = math.ceil(H / patch_size)
    n_cols = math.ceil(W / patch_size)
    for i in range(n_rows):
        for j in range(n_cols):
            y0, y1 = i * patch_size, min((i + 1) * patch_size, H)
            x0, x1 = j * patch_size, min((j + 1) * patch_size, W)
            patch = image[y0:y1, x0:x1, :]
            pad_h, pad_w = patch_size - patch.shape[0], patch_size - patch.shape[1]
            patch_padded = np.pad(patch, ((0, pad_h), (0, pad_w), (0, 0)), mode='reflect')
            pred = model.predict(np.expand_dims(patch_padded, axis=0), verbose=0)
            pred_mask = (pred[0, :, :, 0] > 0.5).astype(np.uint8)
            out_mask[y0:y1, x0:x1] = pred_mask[:patch.shape[0], :patch.shape[1]]
    return out_mask

# === Compute metrics ===
def compute_metrics(y_true, y_pred):
    intersection = np.sum(np.logical_and(y_true, y_pred))
    union = np.sum(np.logical_or(y_true, y_pred))
    iou = intersection / (union + 1e-8)
    dice = (2 * intersection) / (np.sum(y_true) + np.sum(y_pred) + 1e-8)
    report = classification_report(y_true.flatten(), y_pred.flatten(), output_dict=True, zero_division=0)
    return {
        "Accuracy": report["accuracy"],
        "Precision": report["1"]["precision"],
        "Recall": report["1"]["recall"],
        "F1-Score": report["1"]["f1-score"],
        "IoU": iou,
        "Dice": dice
    }

# === Visualization ===
def plot_confusion_matrix(y_true, y_pred, model_name, save_dir):
    cm = confusion_matrix(y_true.flatten(), y_pred.flatten())
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix - {model_name}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    path = os.path.join(save_dir, f"{model_name}_2022_confusion.png")
    plt.savefig(path, dpi=300)
    plt.close()
    print(f"✅ Saved confusion matrix: {path}")

def plot_radar(metrics_dict, model_name, save_dir):
    categories = list(metrics_dict.keys())
    values = list(metrics_dict.values())
    values += values[:1]  # close the loop
    angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()
    angles += angles[:1]

    plt.figure(figsize=(6,6))
    ax = plt.subplot(111, polar=True)
    ax.plot(angles, values, 'o-', linewidth=2, label=model_name)
    ax.fill(angles, values, alpha=0.25)
    ax.set_yticks(np.linspace(0,1,5))
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    plt.title(f"Performance Radar - {model_name}")
    plt.tight_layout()
    save_path = os.path.join(save_dir, f"{model_name}_2022_radar.png")
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"✅ Saved radar plot: {save_path}")

# === Test all models ===
model_names = [
    "unet_2022.keras",
    "resunet_2022.keras",
    "att_unet_2022.keras",
    "att_resunet_2022.keras",
    "asdms_unet_2022.keras"
]

results = {}

for mname in model_names:
    model_path = os.path.join(model_dir, mname)
    if os.path.exists(model_path):
        print(f"\n🚀 Testing {mname} ...")
        model = tf.keras.models.load_model(model_path, compile=False)
        pred_mask = predict_large_image(model, img_2022)
        true_mask = mask_2022[:pred_mask.shape[0], :pred_mask.shape[1]]

        # Metrics
        metrics = compute_metrics(true_mask, pred_mask)
        results[mname.replace("_2022.keras", "")] = metrics

        # Plots
        plot_confusion_matrix(true_mask, pred_mask, mname.replace("_2022.keras", ""), model_dir)
        plot_radar(metrics, mname.replace("_2022.keras", ""), model_dir)

# === Save metrics summary ===
df = pd.DataFrame(results).T.round(4)
csv_path = os.path.join(model_dir, "metrics_summary_2022.csv")
df.to_csv(csv_path)
print(f"\n✅ Metrics saved to CSV: {csv_path}")

# === Bar plot comparison ===
plt.figure(figsize=(10,6))
df.plot(kind='bar', figsize=(10,6))
plt.title("Model Performance Comparison (2022 Data)")
plt.ylabel("Score")
plt.ylim(0,1)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
bar_save_path = os.path.join(model_dir, "barplot_metrics_2022.png")
plt.savefig(bar_save_path, dpi=300)
plt.close()
print(f"✅ Saved bar plot: {bar_save_path}")

print("\n📊 Final Results (All 5 Models on 2022 Data):")
display(df)
